In [1]:
# Import relevant libraries 

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import numpy as np 
import cv2 
import os 

from moviepy.editor import VideoFileClip
# Read in a single image frame 


In [18]:

def preprocessImage(img_frame):
    """ returns a gray scale image filtered with a gaussian"""
    # :: first, convert the image to gray scale 
    imggray = cv2.cvtColor(img_frame, cv2.COLOR_BGR2GRAY)
    img_shape = imggray.shape
    # :: apply a gaussian smoothener to remove noise 
    imggray_blur = cv2.GaussianBlur(imggray,(kernel_size,kernel_size),0)
    
    return imggray_blur

def createROIMask(mask_shape,mask_threshold_color = 255):
    """ returns a mask image where only the pixels within the region 
        of interest is sest to mask_threshold_color"""
    # specify the ROI polygon and set all pixels in that region to 255.
    p1 = (902,537)
    p2 = (533, 326)
    p3 = (434, 326)
    p4 = (131,537)
    
    mask = np.zeros(mask_shape).astype(np.int32)
    vertices = np.array([[p1,p2,p3,p4]])
    cv2.fillPoly(mask, vertices, mask_threshold_color)
    
    # display the ROI polygon on the original image 
    roi_on_original_image = np.copy(img_frame2)
    cv2.polylines(roi_on_original_image,vertices,True,(255,0,0),2)

    return mask

def detectLaneLines(imggray_blur):
    """ returns a list of lines that lie on car lanes """
    # :: apply canny edge detection to detect edges
    img_shape = imggray_blur.shape
    edges = cv2.Canny(imggray_blur,low_threshold,high_threshold)
    # :: create mask to filter out unwanted edges
    mask = createROIMask(img_shape,255)
    print(" edges size = ",edges.shape)
    print("mask size = ",mask.shape)
    print("max edges", edges.max())
    print(edges.min())
    print(mask.max())
    print(mask.min())
    masked_edges = np.bitwise_and(edges,mask)

    # perform hough transform to detect lines, and draw the lines 
    lines = cv2.HoughLinesP(masked_edges, rho, theta, vote_threshold,
                            min_line_length, max_line_gap)
    
    lane_lines = [l[0] for l in lines]
    #:: create a 3-channel form of edges
    color_edges = np.dstack((edges,edges,edges))
    # create a color binary image to combine with line_image 
    if DEBUG_ON:
        line_image = annotateImage(np.zeros_like(color_edges), lane_lines,(255,0,0))
        line_edges = cv2.addWeighted(color_edges,0.8,line_image,1,0)
        plt.figure()
        plt.imshow(line_edges)


    return lane_lines, color_edges

def splitLaneLinesIntoRightAndLeft(lane_lines,split_x_position):
    """ returns an interpolated set of line segments that cover the entire
        span of points in lines """
    # split the lines into right and left line
    left_lines = []
    right_lines = []

    for line in lane_lines: 
        x1,y1,x2,y2 = line
        if (x1 > img_shape[1]*0.5) and (x2 > split_x_position):
            right_lines.append((x1,y1,x2,y2))
        else:
            left_lines.append((x1,y1,x2,y2))

    #:: reformat left and right lines 
    left_x, left_y  = reformatAsCoordinates(left_lines)
    right_x,right_y = reformatAsCoordinates(right_lines)
    
    return left_x, left_y, right_x, right_y
    
def interpolateLaneLines(lane_lines,split_x_position,base_limit):
    #:: split lanes into left and right lanes
    left_x,left_y,right_x,right_y = \
                     splitLaneLinesIntoRightAndLeft(lane_lines,split_x_position)
    # extract points for poly fit 
    left_x_min = min(left_x)
    left_x_max = max(left_x)
    right_x_min = min(right_x)
    right_x_max = max(right_x)
    #:: fit a polynomial of deg 2
    left_line_equ = np.poly1d(np.polyfit(left_x,left_y,2))
    right_line_equ = np.poly1d(np.polyfit(right_x,right_y,2))
    #:: using the fit equation, compute the new lane lines 
    new_left_x = np.linspace(left_x_min, left_x_max,10).astype(np.int32)
    new_left_y = left_line_equ(new_left_x).astype(np.int32)

    new_right_x = np.linspace(right_x_min, right_x_max,10).astype(np.int32)
    new_right_y = right_line_equ(new_right_x).astype(np.int32)
    #:: extend the lane lines to the base of the image 
    #new_left_x, new_left_y = extendLaneLines(new_left_x,new_left_y,base_limit)
    #new_right_x,new_right_y = extendLaneLines(new_right_x,new_right_y,base_limit)
    
    #:: reformat as line segments
    left_lines = reformatAsLineSegments(new_left_x,new_left_y)
    right_lines = reformatAsLineSegments(new_right_x,new_right_y)
    #::
    return left_lines + right_lines

def extendLaneLines(lane_x,lane_y, base_limit):
    """ extends the lane lines to meet the base of the image """
    # 
    pass 

def reformatAsCoordinates(lane_lines):
    """ reformats the list of line segments into a two lists of [x], and [y]"""
    lane_x_points, lane_y_points = [],[]

    for line in lane_lines:
        x1,y1,x2,y2 = line
        lane_x_points.append(x1)
        lane_x_points.append(x2)
        lane_y_points.append(y1)
        lane_y_points.append(y2)
        
    return lane_x_points,lane_y_points

def reformatAsLineSegments(lane_x_points,lane_y_points):
    lane_lines = [] 
    for i in range(len(lane_x_points)-1):
        x1,y1 = lane_x_points[i],lane_y_points[i]
        x2,y2 = lane_x_points[i+1],lane_y_points[i+1]
        lane_lines.append((x1,y1,x2,y2))
    return lane_lines


def annotateImage(img,lane_lines,lane_color):
    """ returns a new image, with the lane lines super-imposed on it """
    line_image = np.zeros_like(img)
    for (x1,y1,x2,y2) in lane_lines:
        cv2.line(line_image,(x1,y1),(x2,y2),lane_color,5)
        
    annotated_image = cv2.addWeighted(img,0.8,line_image,1,0)
    return annotated_image
    

def markLanes(image, lane_color):
    """ returns an image where the lanes are marked with lane_color """
    #:: preprocess image into a sutable form 
    imggray_blur = preprocessImage(image)
    img_shape = imggray_blur.shape 
    #:: detect lane lines
    lane_lines,color_edges = detectLaneLines(imggray_blur)
    #:: interpolate lines for better joining
    new_lane_lines = interpolateLaneLines(lane_lines,img_shape[1]*0.5,
                                                      img_shape[0]*0.9)
    
    if DEBUG_ON:
        interp_line_edges = annotateImage(color_edges,
                                          new_lane_lines,(255,0,0))
        plt.figure()
        plt.imshow(interp_line_edges)
        plt.title('Interpolated edges ')
    #:: superimpose the detected lane lines on the image
    annotated_image = annotateImage(image,new_lane_lines,lane_color)
    #:: return the annotated image 
    return annotated_image 
    


# ==> specify parameters here  
#::kernel size for gaussian blur
kernel_size = 5    
#:: canny edge detector parameters
low_threshold = 30    
high_threshold = 100  
#:: hough line transform parameters
rho = 1               #
theta = np.pi / 180 
vote_threshold = 1 
min_line_length = 6 
max_line_gap = 5
#:: 
mask_threshold_color = 255 
#::
DEBUG_ON = 1

#=> 

# Read in a single image frame and convert to gray scale 
test_img_dir = '.\\test_images\\'
all_img  = [os.path.join(test_img_dir,filename) 
              for filename in os.listdir(test_img_dir) ]

img_frame2 = mpimg.imread(all_img[3])

annotated_img2 = markLanes(img_frame2,(0,255,255))

plt.figure()
plt.imshow(annotated_img2)

 edges size =  (540, 960)
mask size =  (540, 960)
max edges 255
0
255
0


error: C:\projects\opencv-python\opencv\modules\imgproc\src\hough.cpp:425: error: (-215) image.type() == (((0) & ((1 << 3) - 1)) + (((1)-1) << 3)) in function cv::HoughLinesProbabilistic
